In [40]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import keras
import sklearn
import pickle

from keras import utils, layers, models, optimizers

In [7]:
def list_images(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"), contains=None):

    imagePaths = []

    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename).replace(" ", "\\ ")
                imagePaths.append(imagePath)

    return imagePaths

Import data

In [47]:
imagePaths = list_images("data")
labels = []
logos = []
#Boucle qui stocke tous les visages dans data_faces et les noms dans labels
for im in imagePaths :

    image = cv2.imread(im)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if len(imagePaths)>0 :
        logos.append(image)
        if 'BT' in im :
            labels.append([1,0,0,0,0,0])
        elif 'exit' in im :
            labels.append([0,1,0,0,0,0])
        elif 'extincteur' in im :
            labels.append([0,0,1,0,0,0])
        elif 'incendie' in im :
            labels.append([0,0,0,1,0,0])
        elif 'porte' in im :
            labels.append([0,0,0,0,1,0])
        else :
            labels.append([0,0,0,0,0,1])

print(len(labels))
data_dictionaire = (logos,labels)
pickle.dump(data_dictionaire, open("data.p","wb"))

19


In [48]:
from sklearn.model_selection import train_test_split

#Récupere les données de "data.p"
logos , labels = pickle.load(open("data.p",'rb'))

#Normalise les visages et les labels
logos = np.array(logos,dtype=object) / 255
labels = np.array(labels).astype('float32')

#Sépare les données en test et validation
train_images, test_images, train_labels, test_labels  = train_test_split(logos,labels,train_size=0.7, test_size=0.30)

Model

In [ ]:
# Voici le réseau que nous avons décidé d'utiliser dans cet exercice
model = models.Sequential()
model.add( layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)) )
model.add( layers.MaxPooling2D((2, 2)) )
model.add( layers.Conv2D(64, (3, 3), activation='relu') )
model.add( layers.MaxPooling2D((2, 2)) )
model.add( layers.Conv2D(64, (3, 3), activation='relu') )
model.add( layers.MaxPooling2D((2, 2)) )
model.add( layers.Flatten())
model.add( layers.Dense(60, activation='relu'))
model.add(layers.Dropout(0.5)) # On rajoute un dropout pour éviter l'overfitting
model.add(layers.Dense(6, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# Entrainement du réseau
history = model.fit(train_images,
                    train_labels,
                    batch_size=3,
                    epochs=50,
                    validation_data=(test_images,test_labels))

Train on 13 samples, validate on 6 samples
Epoch 1/50
13/13 [==============================] - 2s 138ms/step - loss: 0.5153 - acc: 0.8077 - val_loss: 0.4831 - val_acc: 0.8333
Epoch 2/50
13/13 [==============================] - 1s 55ms/step - loss: 0.4344 - acc: 0.8333 - val_loss: 0.4251 - val_acc: 0.8333
Epoch 3/50
13/13 [==============================] - 1s 56ms/step - loss: 0.3939 - acc: 0.8462 - val_loss: 0.4260 - val_acc: 0.8611
Epoch 4/50
13/13 [==============================] - 1s 54ms/step - loss: 0.2895 - acc: 0.8718 - val_loss: 0.4982 - val_acc: 0.8611
Epoch 5/50
13/13 [==============================] - 1s 56ms/step - loss: 0.2947 - acc: 0.8846 - val_loss: 0.6139 - val_acc: 0.7778
Epoch 6/50
13/13 [==============================] - 1s 55ms/step - loss: 0.3801 - acc: 0.8590 - val_loss: 0.4440 - val_acc: 0.8889
Epoch 7/50
13/13 [==============================] - 1s 53ms/step - loss: 0.2029 - acc: 0.9231 - val_loss: 0.3962 - val_acc: 0.8611
Epoch 8/50
13/13 [=====================